In [1]:
%load_ext lab_black
import pandas as pd
import json


f = open("histohour_DOGECOIN.json",)
data = json.load(f)
data = pd.DataFrame.from_dict(data["Data"]["Data"])
data = data[["time", "close"]]
f.close()
data

,time,close
0,1611691200,0.008199
1,1611694800,0.008164
2,1611698400,0.008220
3,1611702000,0.008269
4,1611705600,0.008177
...,...,...
1996,1618876800,0.370200
1997,1618880400,0.387700
1998,1618884000,0.387300
1999,1618887600,0.405500


In [2]:
def get_profit(lower_limit, upper_limit):
    i = 0
    profit = 0
    records = []
    n = 0
    for index, row in data.iterrows():
        day = index
        price = row["close"]
        if i % 2 == 0:
            if row["RSI"] <= lower_limit and n == 0:
                profit -= price
                records.append(["Buying", -round(price), round(profit), day])
                n += 1
                i += 1
        else:
            if row["RSI"] >= upper_limit and n >= 0 and profit + price >= 1:
                profit += price
                records.append(["Selling", round(price), round(profit), day])
                n -= 1
                i += 1
    return profit


def calculation(data, window_length, lower_limit, upper_limit):
    delta = data["close"].diff()
    up, down = delta.copy(), delta.copy()
    up[up <= 1] = 0
    down[down >= 2] = 0
    roll_up1 = up.ewm(span=window_length).mean()
    roll_down1 = down.abs().ewm(span=window_length).mean()
    RS1 = roll_up1 / roll_down1
    RSI1 = 100.0 - (100.0 / (1.0 + RS1))
    data.loc[:, "RSI"] = RSI1
    data.loc[:, "TimeStamp"] = pd.to_datetime(data["time"], unit="s")
    data = data.set_index("TimeStamp")
    return get_profit(lower_limit, upper_limit)


calculation(data, 14, 50, 60)

-0.008164

In [3]:
%%time
from tqdm import tqdm
optimizations=[]
for window_length in tqdm(range(10, 20)):
    for lower_limit in range(10, 30):
        for upper_limit in range(80, 100):
            profit = calculation(data, window_length, lower_limit,upper_limit)
            optimizations.append([profit, window_length, lower_limit, upper_limit])

100%|██████████| 10/10 [30:53<00:00, 185.40s/it]

Wall time: 30min 54s


In [5]:
optimization_df = pd.DataFrame(
    optimizations, columns=["Profit", "WindowLength", "LowerLimit", "UpperLimit"]
)
pd.set_option("max_rows", 30)
optimization_df

,Profit,WindowLength,LowerLimit,UpperLimit
0,-0.008164,10,10,80
1,-0.008164,10,10,81
2,-0.008164,10,10,82
3,-0.008164,10,10,83
4,-0.008164,10,10,84
...,...,...,...,...
3995,-0.008164,19,29,95
3996,-0.008164,19,29,96
3997,-0.008164,19,29,97
3998,-0.008164,19,29,98


In [6]:
# optimization_df.to_excel("Profit_Optimization_DOGE_hr.xlsx")

In [7]:
optimization_df[optimization_df["Profit"] == optimization_df["Profit"].max()]

,Profit,WindowLength,LowerLimit,UpperLimit
0,-0.008164,10,10,80
1,-0.008164,10,10,81
2,-0.008164,10,10,82
3,-0.008164,10,10,83
4,-0.008164,10,10,84
...,...,...,...,...
3995,-0.008164,19,29,95
3996,-0.008164,19,29,96
3997,-0.008164,19,29,97
3998,-0.008164,19,29,98


In [8]:
window_length = 19
lower_limit = 29
upper_limit = 99

In [9]:
def get_profit(lower_limit, upper_limit):
    i = 0
    profit = 0
    records = []
    n = 0
    for index, row in data.iterrows():
        day = index
        price = row["close"]
        if i % 2 == 0:
            if row["RSI"] <= lower_limit and n == 0:
                profit -= price
                records.append(["Buying", -round(price), round(profit), day])
                n += 1
                i += 1
        else:
            if row["RSI"] >= upper_limit and n >= 0 and profit + price >= 1:
                profit += price
                records.append(["Selling", round(price), round(profit), day])
                n -= 1
                i += 1
    return records, profit


def calculation(data, window_length, lower_limit, upper_limit):
    delta = data["close"].diff()
    up, down = delta.copy(), delta.copy()
    up[up <= 1] = 0
    down[down >= 2] = 0
    roll_up1 = up.ewm(span=window_length).mean()
    roll_down1 = down.abs().ewm(span=window_length).mean()
    RS1 = roll_up1 / roll_down1
    RSI1 = 100.0 - (100.0 / (1.0 + RS1))
    data.loc[:, "RSI"] = RSI1
    data.loc[:, "TimeStamp"] = pd.to_datetime(data["time"], unit="s")
    data = data.set_index("TimeStamp")
    records, profit = get_profit(lower_limit, upper_limit)
    return records, profit


records, profit = calculation(data, window_length, lower_limit, upper_limit)
print(f"Total profit: USD {profit}\n")
records_df = pd.DataFrame(records, columns=["Action", "Price", "Profit", "Day"])
pd.set_option("max_rows", 30)
records_df

Total profit: USD -0.008164



,Action,Price,Profit,Day
0,Buying,0,0,1
